# Brain Tumor Detection
Description
This dataset was originally created by Yousef Ghanem. To see the current project, which may have been updated since this version, please go here: https://universe.roboflow.com/yousef-ghanem-jzj4y/brain-tumor-detection-fpf1f.

This dataset is part of RF100, an Intel-sponsored initiative to create a new object detection benchmark for model generalizability.

Access the RF100 Github repo: https://github.com/roboflow-ai/roboflow-100-benchmark

## Imports

In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/brain-tumor-detection'

In [2]:
from dotenv import load_dotenv
load_dotenv()

import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
tf.config.list_physical_devices('GPU'), tf.__version__

2025-03-16 08:25:38.459395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742113538.468796   95271 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742113538.471274   95271 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742113538.480719   95271 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742113538.480737   95271 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742113538.480739   95271 computation_placer.cc:177] computation placer alr

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.19.0')

In [3]:
# auto reload dotenv 
%load_ext dotenv
%dotenv

# auto reload libs
%load_ext autoreload
%autoreload 2

In [4]:
from src.utils.visualization_funcs import plot_random_images_bbox

## Paths Setup

In [5]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/-Brain-Tumor-Detection-2/train/


In [6]:
cfg.DATASET_DIRS

{'TRAIN_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/train/', 'VALIDATION_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/valid', 'TEST_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/test'}

In [7]:
from pathlib import Path

DATASET_DIRS = Path(cfg.DATASET.DATASET_DIR)
TRAIN_DIR = Path(cfg.DATASET_DIRS.TRAIN_DIR)
TEST_DIR = Path(cfg.DATASET_DIRS.TEST_DIR)

MODEL_CHECKPOINT = Path(cfg.OUTPUTS.CHECKPOINT_PATH)

IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE

CLASS_NAME = [
    'label0',
    'label1',
    'label2'
]
class_map = {v: k for k, v in enumerate(CLASS_NAME)}
class_map[CLASS_NAME[0]]

0

## Dataset Download from Roboflow

In [8]:
if not TRAIN_DIR.exists():
    from roboflow import Roboflow
    rf = Roboflow()
    # https://universe.roboflow.com/roboflow-100/brain-tumor-m2pbp/dataset/2/images?split=test
    project = rf.workspace("roboflow-100").project("brain-tumor-m2pbp")
    version = project.version(2)
    dataset = version.download("tensorflow")

## Load images from directory

In [9]:
from src.data_handler.annotation_processor import AnnotationProcessor
from src.data_handler.data_loader import DataLoader
from src.data_handler.preprocessor import Preprocessor
_class_map = {v: k for k, v in enumerate(CLASS_NAME)}
prepare_test_dataset = AnnotationProcessor(annotation_file= str(TEST_DIR/'_annotations.csv'))
test_image_paths, test_class_ids, test_bboxes  = prepare_test_dataset.process_annotations(image_dir=TEST_DIR, class_id_map=_class_map)
test_dl = DataLoader(test_image_paths, test_class_ids, test_bboxes).load_val_dataset()
test_ds = Preprocessor(test_dl).preprocess()
test_ds = test_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

I0000 00:00:1742113546.728785   95271 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7260 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6


## Load the model

In [10]:
import keras
# model = keras.models.load_model(str(MODEL_CHECKPOINT/'chpt_49.keras'))
# model = keras.models.load_model('output/checkpoints/ckpt_49.keras')

In [12]:
import mlflow
model_uri = 'runs:/62c418bd0b544c06af833620b6b88e8a/model'
model_uri2 = 'runs:/b846a212193641caa4a3d900f2f5dafa/my_model'
loaded_model = mlflow.tensorflow.load_model(model_uri2)

loaded_model.predict(test_ds)

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
# mlflow.models.predict(
#     model_uri=model_uri,
#     input_data=input_data,
#     env_manager="uv",
# )

/code/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 218 variables whereas the saved optimizer has 434 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1742113686.941964   95530 service.cc:152] XLA service 0x7770b808bf70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742113686.941982   95530 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-03-16 08:28:07.038814: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742113687.703687   95530 cuda_dnn.cc:529] Loaded cuDNN version 90300


 5/31 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step

I0000 00:00:1742113689.825803   95530 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 184ms/step


[array([[0.89860874, 0.72601676, 0.9379034 ],
        [0.280414  , 0.6880886 , 0.6536545 ],
        [0.9940362 , 0.7807087 , 0.9924979 ],
        ...,
        [0.94847995, 0.6151853 , 0.8396786 ],
        [0.8802551 , 0.66000146, 0.80798393],
        [0.9801492 , 0.8517702 , 0.9924575 ]], dtype=float32),
 array([[[0.23847501, 0.15602894, 0.7497942 , 0.97365856],
         [0.23101969, 0.6446964 , 0.8395523 , 0.9782326 ],
         [0.09746745, 0.5666144 , 0.6985348 , 0.94507325]],
 
        [[0.31629896, 0.10791071, 0.8833301 , 0.6657304 ],
         [0.35961854, 0.1489991 , 0.9143051 , 0.59768575],
         [0.27483326, 0.15443334, 0.8167001 , 0.62916225]],
 
        [[0.2605072 , 0.06173887, 0.5683742 , 0.99473315],
         [0.19748934, 0.79036164, 0.8984598 , 0.9979494 ],
         [0.02487221, 0.8128542 , 0.7176682 , 0.9916182 ]],
 
        ...,
 
        [[0.05268292, 0.10238665, 0.3971132 , 0.58747256],
         [0.01898773, 0.12334145, 0.4721198 , 0.70663345],
         [0.04234122,

In [ ]:
results = model.evaluate(test_ds, return_dict=True)
results

In [ ]:
unnorm_bbx = []
for bbx in test_bboxes:
   unnorm_bbx.append(bbx*IMG_SIZE)

In [ ]:
import random


def plot_random_images_bbox(*,  random_samples:np.ndarray, image_paths:np.ndarray, class_ids:np.ndarray, bboxes:np.ndarray, class_map:dict) -> None:
  fig = plt.figure(figsize=(8, 8))

  print(f"Random samples: {random_samples}")
  class_map_invert = {v: k for k, v in class_map.items()}
  
  for i, idx in enumerate(random_samples):
    ax = fig.add_subplot(3, 3, i+1)
    image = image_paths[idx]
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Create title from class IDs
    title_labels = [class_map_invert[int(cls_id)] for cls_id in class_ids[idx]]
    title = ", ".join(title_labels)
    ax.set_title(title)
    ax.imshow(image) #display image before bounding box

    # Draw bounding boxes with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0,255,255), (255,0,255)] # Example colors
    for j, (xmin, ymin, xmax, ymax) in enumerate(bboxes[idx]):
        color = colors[j % len(colors)] # Cycle through colors
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 1)
    ax.imshow(image) #display image with bounding box.

  plt.tight_layout() #prevents overlapping subplots
  plt.show()

In [ ]:
random_samples = random.sample(range(len(test_images)), 9)
plot_random_images_bbox(random_samples=random_samples,
                        image_paths=test_images, 
                        class_ids=test_class_ids, 
                        bboxes=unnorm_bbx,
                        class_map=class_map)

In [ ]:
pred_cls_id, pred_bbx = model.predict(test_ds)
print(pred_cls_id.shape, pred_bbx.shape)

In [ ]:
pred_bbx

In [ ]:
unnorm_pred_bbx = []
for bbx in pred_bbx:
   unnorm_pred_bbx.append(bbx*IMG_SIZE)

In [ ]:
pred_cls_id = (pred_cls_id>0.5).astype(int)
pred_cls_id

In [ ]:
pred_cls = [np.where(row==1)[0].tolist() for row in pred_cls_id]
pred_cls

In [ ]:
for idx in random_samples:
   print([cls_id for cls_id in pred_cls[idx]])

In [ ]:
plot_random_images_bbox(random_samples=random_samples,
                        image_paths=test_images, 
                        class_ids=pred_cls, 
                        bboxes=unnorm_pred_bbx,
                        class_map=class_map)

In [ ]:
unnorm_bbx, pred_bbx

In [ ]:
from src.losses.iou_loss import iou_metric
for y_tbbx, y_prdbbx in zip(unnorm_bbx, unnorm_pred_bbx):
    print(iou_metric(y_true=y_tbbx, y_pred=y_prdbbx)) 